This is the jupiter notebook that Zhou writes some test functions

In [ ]:
from functions import *
from optimize_obj import *
from Optalg import *
from runcase import *
import pandas as pd

In [ ]:
def generate_quad_coeffs(n, k):
    # n is dimension
    # k is condition number
    np.random.seed(0)
    Q = np.random.rand(n, n)
    Q, _ = np.linalg.qr(Q)
    eigenvalues = np.linspace(1, k, n)
    D = np.diag(eigenvalues)
    Q = Q @ D @ Q.T

    # Ensure Q is positive definite and the condition number is approximately k
    assert np.all(np.linalg.eigvals(Q) > 0), "Matrix Q is not positive definite."
    assert np.isclose(np.linalg.cond(Q), k), "Condition number of matrix Q is not approximately k."

    q = np.random.rand(n)

    return Q, q

This is for the problem 1

In [ ]:
Q1, q1 = generate_quad_coeffs(10, 10)

# generate 10 random numbers between 0, 1
np.random.seed(0)
x0 = 20 * np.random.rand(10) - 10
#obj = quad_obj(Q1, q1)
obj = rosenbrock_obj(10)
x0 = np.ones(10) * 2.2

runcase(obj, x0, Objname = 'p1',make_table = True)
#runcase(obj, x0, Objname = 'p1', line_search_method = 'Wolfe', make_table = True)

In [ ]:
obj.method_options.methodname

This is for the problem 2

In [4]:
Q2, q2 = generate_quad_coeffs(10, 1000)
x0 = 20 * np.random.rand(10) - 10
obj = quad_obj(Q2, q2)

runcase(obj, x0, Objname = 'p2', make_table = True)
#runcase(obj, x0, Objname = 'p2', line_search_method = 'Wolfe', make_table = True)

TypeError: runcase() got an unexpected keyword argument 'line_search_method'

This is for the problem 3

In [ ]:
Q3, q3 = generate_quad_coeffs(1000, 10)
x0 = 20 * np.random.rand(1000) - 10
obj = quad_obj(Q3, q3)

runcase(obj, x0, Objname = 'p3', make_table = True)
#runcase(obj, x0, Objname = 'p3', line_search_method = 'Wolfe', make_table = True)

This is for the problem 4

In [ ]:
Q4, q4 = generate_quad_coeffs(1000, 1000)
x0 = 20 * np.random.rand(1000) - 10
obj = quad_obj(Q4, q4)

runcase(obj, x0, Objname = 'p4', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p4', line_search_method = 'Wolfe', make_table = True)

This is for the problem 5

In [ ]:
# Problem 5

Q = np.array([[5, 1, 0, 0.5], [1, 4, 0.5, 0], [0, 0.5, 3, 0], [0.5, 0, 0, 2]])
sigma = 0.0001
x0 = np.array([np.cos(70), np.sin(70), np.cos(70), np.sin(70)])
obj = special_quad_obj(Q, sigma)

runcase(obj, x0, Objname = 'p5', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p5', line_search_method = 'Wolfe', make_table = True)

this is for the problem 6

In [ ]:
# Problem 6

Q = np.array([[5, 1, 0, 0.5], [1, 4, 0.5, 0], [0, 0.5, 3, 0], [0.5, 0, 0, 2]])
sigma = 10000
x0 = np.array([np.cos(70), np.sin(70), np.cos(70), np.sin(70)])
obj = special_quad_obj(Q, sigma)

runcase(obj, x0, Objname = 'p6', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p6', line_search_method = 'Wolfe', make_table = True)

this is for the problem 7

In [ ]:
# Problem 7

obj = rosenbrock_obj(2)
x0 = np.array([-1.2, 1])

runcase(obj, x0, Objname = 'p7', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p7', line_search_method = 'Wolfe', make_table = True)

this is for the problem 8

In [ ]:
obj = rosenbrock_obj(100)
x0 = np.array([-1.2] + [1]*99)

runcase(obj, x0, Objname = 'p8', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p8', line_search_method = 'Wolfe', make_table = True)

this is for the problem 9

In [ ]:
y = np.array([1.5, 2.25, 2.625])
obj = data_fit_obj(y)
x0 = np.array([1, 1])

runcase(obj, x0, Objname = 'p9', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p9', line_search_method = 'Wolfe', make_table = True)

this is for the problem 10

In [ ]:
obj = exponential_obj(10)
x0 = np.array([1] + [0]*9)

runcase(obj, x0, Objname = 'p10', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p10', line_search_method = 'Wolfe', make_table = True)

this is for the problem 11

In [ ]:
obj = exponential_obj(100)
x0 = np.array([1] + [0]*99)

runcase(obj, x0, Objname = 'p11', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p11', line_search_method = 'Wolfe', make_table = True)


this is for the problem 12

In [ ]:
obj = genhump_obj(5)
x0 = np.array([-506.2, 506.2, 506.2, 506.2, 506.2])

runcase(obj, x0, Objname = 'p12', line_search_method = 'Armijo', make_table = True)
runcase(obj, x0, Objname = 'p12', line_search_method = 'Wolfe', make_table = True)

The following code is for extracting the txt file, and plot the function value and gradient value vs iteration

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Define the list of methods and line searches
method_list = ['BFGS', 'gradient_descend', 'L-BFGS', 'modified_Newton']
line_search_list = ['Armijo', 'Wolfe']

def plot_for_problem(i):
    # Setup the figure for this problem with subplots for function value and gradient norm
    fig, axs = plt.subplots(4, 4, figsize=(24, 16), sharex='col')  # 4 rows, 4 columns
    axs = axs.flatten()
    
    # Title for the entire figure
    plt.suptitle(f'Performance Comparison for Problem {i}', fontsize=16)

    for j, (method, line_search) in enumerate([(m, ls) for m in method_list for ls in line_search_list]):
        try:
            # Load the result data
            df = pd.read_csv(f'result/p{i}_{method}_{line_search}.txt', delim_whitespace=True)
            # Log scale transformations to handle large variations in values
            df['fval'] = np.log(df['fval'].replace(0, np.nan))
            df['||grad||'] = np.log(df['||grad||'].replace(0, np.nan))

            # Function Value Plot
            ax_fval = axs[2*j]  # Even index for function values
            ax_fval.plot(df['Iter'], df['fval'], label=f'{method}, {line_search}')
            ax_fval.set_title(f'{method} + {line_search} (Fval)')
            ax_fval.set_ylabel('Log Function Value')
            ax_fval.legend()

            # Gradient Norm Plot
            ax_grad = axs[2*j + 1]  # Odd index for gradient norms
            ax_grad.plot(df['Iter'], df['||grad||'], label=f'{method}, {line_search}', color='red')
            ax_grad.set_title(f'{method} + {line_search} (Grad Norm)')
            ax_grad.set_ylabel('Log Gradient Norm')
            ax_grad.legend()

        except FileNotFoundError:
            axs[2*j].set_title('Data Not Found')
            axs[2*j].set_visible(False)
            axs[2*j + 1].set_title('Data Not Found')
            axs[2*j + 1].set_visible(False)

    # Adjust layout and save the figure
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.savefig(f'Problem_{i}_Comparison.png')  # Save figure to PNG file
    plt.show()

# Generate plots for each problem
for i in range(1, 13):  # Assuming there are 12 problems
    plot_for_problem(i)
